<a href="https://colab.research.google.com/github/kuds/rl-car-racing/blob/main/%5BCar%20Racing%5D%20Proximal%20Policy%20Optimization%20(PPO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install swig moviepy

In [2]:
!pip install gymnasium gymnasium[box2d] stable_baselines3 rl_zoo3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.5 MB/s eta 0:00:00


In [1]:
import gymnasium
from gymnasium.wrappers import RecordVideo

import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecFrameStack, VecVideoRecorder
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.vec_env import VecTransposeImage
from stable_baselines3.common.evaluation import evaluate_policy

import platform

import torch
import time
torch.backends.cudnn.benchmark = True

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
print("Is Cuda Available: {}".format(torch.cuda.is_available()))
print("Torch Version: {}".format(torch.__version__))
print("Cuda Version: {}".format(torch.version.cuda))
print("Stable Baseline Version: {}".format(stable_baselines3.__version__))
print("Gymnasium Version: {}".format(gymnasium.__version__))
print("Python Version: {}".format(platform.python_version()))

Is Cuda Available: True
Torch Version: 2.3.1+cu121
Cuda Version: 12.1
Stable Baseline Version: 2.3.2
Gymnasium Version: 0.29.1
Python Version: 3.10.12


In [5]:
# Create CarRacing environment

#env = gymnasium.make("CarRacing-v2", render_mode="rgb_array")
env = make_vec_env("CarRacing-v2", n_envs=32)
env = VecFrameStack(env, n_stack=4)
#env = VecTransposeImage(env)

#env_val = gymnasium.make("CarRacing-v2", render_mode="rgb_array")
env_val = make_vec_env("CarRacing-v2", n_envs=1)
env_val = VecFrameStack(env_val, n_stack=4)
env_val = VecTransposeImage(env_val)

eval_callback = EvalCallback(env_val, best_model_save_path="./logs/", log_path="./logs/", eval_freq=1000, deterministic=True, render=False)

# Initialize PPO
model = PPO('CnnPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=1000000,  progress_bar=True, callback=eval_callback)

# Save the model
model.save("ppo_car_racing")

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

Using cuda device
Wrapping the env in a VecTransposeImage.


Output()

Eval num_timesteps=3200, episode_reward=-31.17 +/- 2.07

Episode length: 1000.00 +/- 0.00

---------------------------------
| eval/              |          |
|    mean_ep_length  | 1e+03    |
|    mean_reward     | -31.2    |
| time/              |          |
|    total_timesteps | 3200     |
---------------------------------


...

New best mean reward!

Eval num_timesteps=12800, episode_reward=-18.70 +/- 20.57

Episode length: 1000.00 +/- 0.00

---------------------------------
| eval/              |          |
|    mean_ep_length  | 1e+03    |
|    mean_reward     | -18.7    |
| time/              |          |
|    total_timesteps | 12800    |
---------------------------------


KeyboardInterrupt: 

In [3]:
# prompt: Write code to get the best model and run gymnasium's Car racing against it. Save the results to a gif

# Load the best model
env = make_vec_env("CarRacing-v2", n_envs=1)
env = VecFrameStack(env, n_stack=4)
env = VecTransposeImage(env)
best_model_path = "./logs/best_model.zip"
best_model = PPO.load(best_model_path, env=env)

# Record video of the best model playing CarRacing
env = VecVideoRecorder(env, "./videos/", record_video_trigger=lambda x: x == 0, video_length=1000, name_prefix="best_model_car_racing")

obs = env.reset()
for _ in range(1000):
    action, _states = best_model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
    env.render()

env.close()

Saving video to /content/videos/best_model_car_racing-step-0-to-step-1000.mp4
Moviepy - Building video /content/videos/best_model_car_racing-step-0-to-step-1000.mp4.
Moviepy - Writing video /content/videos/best_model_car_racing-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/best_model_car_racing-step-0-to-step-1000.mp4
